# SCIERC format

### Libraries and variables

In [1]:
import pandas as pd
import numpy as np

### Process data

In [2]:
def format_scierc(df):
    df=df.assign(text=df.title_abstract_clean)

    df.loc[:,['text']]=df.loc[:,['text']].replace(",", " , ",regex=True).replace("\.", " . ",regex=True).replace(":", " : ",regex=True).replace("-", " - ",regex=True).replace("  ", " ",regex=True)
    df.text=df.text.str.rstrip()

    df=df.assign(sentences=df.text.str.split('.'))
    df=df.loc[:,['ID','sentences']].rename(columns={'ID':'doc_key'})
    df['ner']=[list([list([])]) for _ in range(df.shape[0])]
    df['clusters']=[list([list([])]) for _ in range(df.shape[0])]
    df['relations']=[list([list([])]) for _ in range(df.shape[0])]
    for i,d in df.iterrows():
        sents=[]
        ner=[]
        for s in d['sentences']:
            if s not in [' ','  ','   ']:
                sents.append(s.split())
                ner.append(list())
        sents=sents[:-1]
        ner=ner[:-1]
        df.loc[i,'sentences']=sents
        df.at[i,'ner']=list(ner)
        df.at[i,'relations']=list(ner)
        df.at[i,'clusters']=list(ner)

    return df

In [ ]:
def main():
    data_path="../../data/"
    output_path="../../outputs/"

    df=pd.read_csv(output_path+"sg_ie/positives_ready.csv")
    test_set_complete=pd.read_csv(output_path+"general/test_set_final.csv")
    train_set=pd.read_csv(output_path+"general/train_set_final.csv")
    dev_set=pd.read_csv(output_path+"general/dev_set_final.csv")
    df_test=pd.concat([test_set_complete,dev_set,train_set]).reset_index(drop=True)
    df_test=df_test.loc[df_test.label==1].reset_index(drop=True)
    df_words=format_scierc(df)
    df_test_words=format_scierc(df_test)
    df_test_words=df_test_words.loc[~df_test_words.doc_key.isin(['yu-2007-tai'])]

    df_words.to_json(output_path+"sg_ie/sg_papers_scierc.json",orient="records",lines=True)

    df_test_words.to_json(output_path+"sg_ie/sg_papers_scierc_test.json",orient="records",lines=True)
if __name__ == '__main__':
    main()